In [1]:

# coding: utf-8

# Check [pytesseract](https://pypi.org/project/pytesseract/).

# In[13]:


from pytesseract import *
import os
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re

filename = os.path.join(os.getcwd(), '../sample1/UCD_Lehmann_0015.jpg')
    



In [5]:
filename = os.path.join(os.getcwd(), '../sample1/UCD_Lehmann_0015.jpg')

In [6]:
img = cv2.imread(filename)
img1 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# img2 = cv2.Laplacian(img, cv2.CV_64F)
out = image_to_data(img1)

csv_file = out.replace('\t', ',')
with open('tmp.csv', "w") as f:
    f.write(csv_file)

df = pd.read_csv('tmp.csv', sep=',', error_bad_lines=False, engine="python")
df = df[df.text.notnull()].reset_index(drop=True)

df1 = df.text.apply(lambda x: bool(re.match(r"\d+\.\d{2}", x)), 1)
df1 = df[df1].reset_index()
df1 = df1.rename(columns={'index': 'old_index'})
df1['begin'] = 0
print(df1.head())
df2 = df.text.apply(lambda x: bool(re.match(r"[0-9]+$", x)), 1)
df2 = df[df2].reset_index()
df2 = df2.rename(columns={'index': 'old_index'})
df2['begin'] = 1

df3 = pd.concat([df1, df2]).sort_values('old_index').reset_index(drop=True)
# df3.head(20)

   old_index  level  page_num  block_num  par_num  line_num  word_num  left  \
0        104      5         1          8        1         2         4  1586   
1        105      5         1          8        1         2         5  1797   
2        149      5         1         11        1         2         4  3331   
3        150      5         1         11        1         2         5  3539   
4        202      5         1         15        1         2        18  2701   

    top  width  height  conf   text  begin  
0  1543    110      42    89   1.99      0  
1  1543    141      42    88  21.50      0  
2  1538    109      42    89   4.29      0  
3  1539    139      42    88  46.35      0  
4  1655     78      29    88   2.29      0  


Skipping line 121: Expected 12 fields in line 121, saw 13
Skipping line 138: Expected 12 fields in line 138, saw 13
Skipping line 183: Expected 12 fields in line 183, saw 13
Skipping line 206: Expected 12 fields in line 206, saw 13
Skipping line 209: Expected 12 fields in line 209, saw 13
Skipping line 275: Expected 12 fields in line 275, saw 13
Skipping line 306: Expected 12 fields in line 306, saw 13
Skipping line 350: Expected 12 fields in line 350, saw 13
Skipping line 364: Expected 12 fields in line 364, saw 13
Skipping line 376: Expected 12 fields in line 376, saw 13
Skipping line 398: Expected 12 fields in line 398, saw 13
Skipping line 399: Expected 12 fields in line 399, saw 13
Skipping line 400: Expected 12 fields in line 400, saw 13
Skipping line 420: Expected 12 fields in line 420, saw 13
Skipping line 492: Expected 12 fields in line 492, saw 13
Skipping line 509: Expected 12 fields in line 509, saw 13
Skipping line 553: Expected 12 fields in line 553, saw 13
Skipping line 

In [12]:
df3 = pd.concat([df1, df2]).sort_values('old_index').reset_index(drop=True)
df3.head(20)

,old_index,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,begin
0,2,5,1,3,1,1,1,385,748,656,163,82,1964,1
1,95,5,1,7,1,1,2,381,1407,28,29,90,0,1
2,98,5,1,8,1,1,1,336,1472,93,41,88,842,1
3,104,5,1,8,1,2,4,1586,1543,110,42,89,1.99,0
4,105,5,1,8,1,2,5,1797,1543,141,42,88,21.50,0
5,142,5,1,11,1,1,1,2088,1474,91,40,90,271,1
6,149,5,1,11,1,2,4,3331,1538,109,42,89,4.29,0
7,150,5,1,11,1,2,5,3539,1539,139,42,88,46.35,0
8,183,5,1,15,1,1,29,3913,1616,24,44,71,36,1
9,202,5,1,15,1,2,18,2701,1655,78,29,88,2.29,0


In [13]:
res = []
for i in range(len(df3) - 1):
    # for i in range(5):
    if df3.iloc[i].begin == 1 and (i + 1 == len(df3) or df3.iloc[i + 1].begin == 0):
        # print()
        # print("start at:", df3.iloc[i]['old_index'], df3.iloc[i].text)
        tmp = [None] * 6  # [No.,name, price1, price2, price3, description]
        tmp[0] = df3.iloc[i].text
        start = i

        # prices:
        end = i + 1
        
        while end < len(df3) and df3.iloc[end].begin == 0:
            print(end, end - start + 1)
            tmp[end - start + 1] = df3.iloc[end].text
            end += 1
        # print("end at:", df3.iloc[end-1]['old_index'])

        # name:
        s = ''
        for k in range(int(df3.iloc[i]['old_index']) + 1, int(df3.iloc[i + 1]['old_index'])):
            #print(k, df.iloc[k].text)
            s += ' ' + df.iloc[k].text
        # print(s)
        s = s.rstrip(' .')
        tmp[1] = s

        # description
        s = ''
        for k in range(int(df3.iloc[end - 1]['old_index']) + 1, int(df3.iloc[end]['old_index'])):
            s += ' ' + df.iloc[k].text
        # print(s)
        s = s.rstrip(' .')
        tmp[-1] = s

        res.append(tmp)

res_df = pd.DataFrame(np.array(res), columns=["No.", "Name", "Price1", "Price2", "Price3", "Description"])
res_df.head()
res_df.to_csv(filename[:-3] + "csv")

3 2
4 3
6 2
7 3
9 2
10 3
13 2
14 3
15 4
16 5
17 6


IndexError: list assignment index out of range

In [14]:
df3.head(20)

,old_index,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,begin
0,2,5,1,3,1,1,1,385,748,656,163,82,1964,1
1,95,5,1,7,1,1,2,381,1407,28,29,90,0,1
2,98,5,1,8,1,1,1,336,1472,93,41,88,842,1
3,104,5,1,8,1,2,4,1586,1543,110,42,89,1.99,0
4,105,5,1,8,1,2,5,1797,1543,141,42,88,21.50,0
5,142,5,1,11,1,1,1,2088,1474,91,40,90,271,1
6,149,5,1,11,1,2,4,3331,1538,109,42,89,4.29,0
7,150,5,1,11,1,2,5,3539,1539,139,42,88,46.35,0
8,183,5,1,15,1,1,29,3913,1616,24,44,71,36,1
9,202,5,1,15,1,2,18,2701,1655,78,29,88,2.29,0


In [7]:
res_df.head()

,No.,Name,Price1,Price2,Price3,Description
0,1978,(Sonoma),5.99,64.75,None,
1,1978,(North Coast),5.99,68.75,None,
2,1978,(California),6.39,69.00,None,
3,1979,(Monterey),6.95,75.10,None,_
4,1977,CHALK HILL VINEYARDS (Sonoma),7.49,80.95,None,Rodney Strong's Chardonnay is rich and full. (i
